In [1]:
import sys
sys.path.append('Scripts')
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets, models
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np

from utils import path
from DataVis import createInputFigure
from NNUtils import *
from NNCreationAux import *

from sklearn import preprocessing
import os
import time
import copy

***

In [ ]:
modelType = 'resnet'
modelSize = '101'
data_type = '512'
figSize = '128'
folder = 'GAF_Combinados_difference'

model_name = modelType+modelSize

In [ ]:
dataloader, dataloaders, dataset_sizes, class_names = getData(data_type, folder)

In [ ]:
model = model_selection(modelName, numClasses)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma = 0.1)
model, stats = train(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, num_epochs = 2)

***

In [ ]:
path = 'C:\\Users\\Patrick\\Documents\\GitHub\\ICDeepLearning\\Data\\GAF_Combinados_summation\\512'
dataset = get_dataset(path, 224)
data_type = '512'
folder = 'GAF_Combinados_difference'
dataloader, dataloaders, dataset_sizes, class_names = getData(data_type, folder, batch_size = 1)

In [3]:
# model = models.resnet101(pretrained = True)
# model = model.to(device)
res18 = model_selection('resnet18', 5).to(device)
optimizer = optim.SGD(res18.parameters(), lr = 0.001, momentum = 0.9)
criterion = nn.CrossEntropyLoss()
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 7, gamma = 0.1)

In [4]:
model, _, _ = kfold_train(res18, dataset, criterion, optimizer, num_epochs = 10)

Fold 0
--
Epoch 1/10
-----
Stage: train
Loss: 1.8969 - Accuracy: 0.2459
Stage: test
Loss: 1.7930 - Accuracy: 0.2985
--
Epoch 2/10
-----
Stage: train
Loss: 1.7248 - Accuracy: 0.3246
Stage: test
Loss: 1.7181 - Accuracy: 0.3443
--
Epoch 3/10
-----
Stage: train
Loss: 1.4823 - Accuracy: 0.4386
Stage: test
Loss: 1.2716 - Accuracy: 0.5201
--
Epoch 4/10
-----
Stage: train
Loss: 1.1772 - Accuracy: 0.5737
Stage: test
Loss: 1.2095 - Accuracy: 0.5769
--
Epoch 5/10
-----
Stage: train
Loss: 0.8228 - Accuracy: 0.7001
Stage: test
Loss: 1.1554 - Accuracy: 0.6593
--
Epoch 6/10
-----
Stage: train
Loss: 0.6998 - Accuracy: 0.7660
Stage: test
Loss: 1.0890 - Accuracy: 0.6941
--
Epoch 7/10
-----
Stage: train
Loss: 0.5813 - Accuracy: 0.8077
Stage: test
Loss: 1.0261 - Accuracy: 0.7509
--
Epoch 8/10
-----
Stage: train
Loss: 0.5134 - Accuracy: 0.8283
Stage: test
Loss: 1.0055 - Accuracy: 0.7473
--
Epoch 9/10
-----
Stage: train
Loss: 0.4350 - Accuracy: 0.8466
Stage: test
Loss: 1.2397 - Accuracy: 0.7088
--
Epoch 10/

***

In [5]:
batch_size = 1
image_size = 224
folder = 'Test_GASF'
figPath = 'C:\\Users\\Patrick\\Documents\\GitHub\\ICDeepLearning\\Data\\'+folder+'\\512'
dataloader, dataloaders, dataset_sizes, class_names = getData(data_type, folder, batch_size = 1)

In [6]:
class predictor(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        
    def predict(self, dataloader):
        outputs = []
        targets = []
        for sample, target in dataloader:
            sample = sample.to(device)
            target = target.to(device)
            o = torch.argmax(F.softmax(self.model(sample), dim = 1)).item()
            outputs.append(o)
            targets.append(target.item())
            
        outputs = np.array(outputs)
        targets = np.array(targets)
        right_pred = 1*(outputs == targets)
        return outputs, right_pred

In [14]:
predic = predictor(res18)
_, outres18 = predic.predict(dataloader)

In [15]:
predic = predictor(res34)
_, outres34 = predic.predict(dataloader)

In [7]:
predic = predictor(model)
_, outmodel = predic.predict(dataloader)

In [8]:
np.count_nonzero(outmodel)/500

0.188

In [19]:
np.count_nonzero(outres18)/500

0.178

In [20]:
np.count_nonzero(outres34)/500

0.202

In [22]:
from scipy import stats
stats.ttest_ind(outres18, outres34, equal_var=True)

Ttest_indResult(statistic=-0.9667862845385763, pvalue=0.33388509993212834)